## Notebook MySQL
En este notebook realizaremos la conexion e insercion de datos en mysql, ademas de las consultas e visualizacion de datos

In [1]:
import mysql.connector
import time
import pandas as pd
import random
import logging
from faker import Faker

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# Funcion para medir el tiempo de ejecucion de una funcion
def medir_tiempo_ejecucion(func, *args, **kwargs):
    """Mide el tiempo de ejecucion de una funcion."""
    inicio = time.time()  # Guardar el tiempo de inicio
    resultado = func(*args, **kwargs)  # Ejecutar la funcion
    fin = time.time()  # Guardar el tiempo de fin
    tiempo_ejecucion = inicio - fin 
    print(f"Tiempo de ejecucion: {tiempo_ejecucion:.5f} segundos")
    return resultado

In [4]:
# Inicializar Faker
fake = Faker()

# Funcion para generar el nombre
def generate_random_name():
    return fake.first_name()

# Funcion para generar el apellido
def generate_random_last_name():
    return fake.last_name()

# Funcion para generar el email
def generate_random_email():
    return fake.email()

# Funcion para generar el genero
def generate_random_gender():
    return random.choice(['Male', 'Female', 'Other'])

# Generar el millon de registros
data = {
    'id': [],
    'first_name': [],
    'last_name': [],
    'email': [],
    'gender': []
}

for i in range(22, 100001):
    data['id'].append(i)
    data['first_name'].append(generate_random_name())
    data['last_name'].append(generate_random_last_name())
    data['email'].append(generate_random_email())
    data['gender'].append(generate_random_gender())
    
    # Loguear cada 100,000 registros
    if i % 100000 == 0:
        logging.info(f"Generados {i} registros")

# Crear DataFrame
df = pd.DataFrame(data)

# Guardar DataFrame en un archivo CSV
df.to_csv('/data/MOCK_DATA_LONG.csv', index=False)

logging.info("Generado correctamente archivo con 1 millon de registros")

2024-11-17 12:59:17,113 - INFO - Generados 100000 registros
2024-11-17 12:59:17,321 - INFO - Generado correctamente archivo con 1 millon de registros


In [4]:
def wait_for_mysql(): # Intentar conectarse a MySQL
    while True:
        try:
            connection = mysql.connector.connect(
                host='mysql',
                user='root',
                password='rootpassword',
                connection_timeout=10
            )
            cursor = connection.cursor()
            print("Conexion exitosa a MySQL")
            return connection, cursor
        except Exception as e:
            print(f"Cargando MySQL... {e}")
            time.sleep(30)

In [5]:
def insert_data_from_csv(connection, cursor, file_path): # Insertar los datos del CSV
    try:
        df = pd.read_csv(file_path)
        print("Archivo CSV leido correctamente")

        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO users (id, first_name, last_name, email, gender)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['id'], row['first_name'], row['last_name'], row['email'], row['gender']))
            print(f"Insertado registro: {row.to_dict()}")

        connection.commit()  # Asegurar que los datos se guarden
        print("Datos insertados correctamente")

    except Exception as e:
        print(f"Error al procesar el archivo CSV: {e}")

In [6]:
def long_data_insert(connection, cursor):
    insert_data_from_csv(connection, cursor, '/data/MOCK_DATA_LONG.csv')
    rows = cursor.execute("SELECT * FROM user_data.users")
    rows = cursor.fetchall() # El metodo execute de cursor no devuelve directamente las filas de la consulta, sino el numero de filas afectadas
    df_rows = pd.DataFrame(list(rows))
    print(df_rows.head())


if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    try:
        medir_tiempo_ejecucion(long_data_insert, connection, cursor)
    finally:
        cursor.close()
        connection.close()

Conexion exitosa a MySQL
Archivo CSV leido correctamente
Error al procesar el archivo CSV: 1046 (3D000): No database selected
   0         1         2                        3       4
0  1    Athena   Elijahu  aelijahu0@moonfruit.com  Female
1  2   Selinda  Paladini       spaladini1@who.int  Female
2  3  Charmain     Sousa        csousa2@imgur.com  Female
3  4     Perry   Harvett       pharvett3@uiuc.edu    Male
4  5     Debbi    Seeler        dseeler4@usda.gov  Female
Tiempo de ejecucion: -0.06729 segundos


In [12]:
def fetch_all_users(connection, cursor): # Devolver todos los usuarios
    try:
        cursor.execute("SELECT * FROM user_data.users LIMIT 10")
        results = cursor.fetchall()
        
        print("Datos en la tabla 'users':")
        for row in results:
            print(row)
    except Exception as e:
        print(f"Error al recuperar datos: {e}")

if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        fetch_all_users(connection, cursor)  # Llamada a la funcion para comprobar los datos
    finally:
        cursor.close()
        connection.close()

Conexion exitosa a MySQL
Datos en la tabla 'users':
(1, 'Athena', 'Elijahu', 'antonlago04@example.com', 'Female')
(2, 'Selinda', 'Paladini', 'spaladini1@who.int', 'Female')
(3, 'Charmain', 'Sousa', 'csousa2@imgur.com', 'Female')
(4, 'Perry', 'Harvett', 'pharvett3@uiuc.edu', 'Male')
(5, 'Debbi', 'Seeler', 'dseeler4@usda.gov', 'Female')
(6, 'Reinhold', 'Wilcockes', 'rwilcockes5@chicagotribune.com', 'Male')
(7, 'Tarah', 'Quilkin', 'tquilkin6@usatoday.com', 'Female')
(8, 'Maddie', 'Mion', 'mmion7@issuu.com', 'Male')
(9, 'Alfie', 'Gobel', 'agobel8@yahoo.co.jp', 'Female')
(10, 'Nadean', 'Paulich', 'npaulich9@wikispaces.com', 'Female')


In [7]:
def get_user_by_id(connection, cursor, user_id):
    try:
        cursor.execute("""
            SELECT * FROM user_data.users WHERE id = %s
        """, (user_id,))
        result = cursor.fetchone()
        if result:
            # Desempaquetar la tupla directamente
            id, first_name, last_name, email, gender = result
            print(f"ID: {id}, Nombre: {first_name} {last_name}, Email: {email}, Genero: {gender}")
        else:
            print("Usuario no encontrado.")
    
    except Exception as e:
        print(f"Error al recuperar el usuario: {e}")

In [8]:
def update_user_email(connection, cursor, user_id, new_email):
    try:
        cursor.execute("""
            UPDATE user_data.users
            SET email = %s
            WHERE id = %s
        """, (new_email, user_id))
        connection.commit()  # Añadir el commit para guardar los cambios
        print(f"Email actualizado para el usuario con ID {user_id}.")
    except Exception as e:
        print(f"Error al actualizar el email: {e}")

if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        medir_tiempo_ejecucion(update_user_email, connection, cursor, 1, "antonlago04@example.com")
        medir_tiempo_ejecucion(get_user_by_id, connection, cursor, 1)
    finally:
        cursor.close()
        connection.close()

Conexion exitosa a MySQL
Email actualizado para el usuario con ID 1.
Tiempo de ejecucion: -0.00693 segundos
ID: 1, Nombre: Athena Elijahu, Email: antonlago04@example.com, Genero: Female
Tiempo de ejecucion: -0.00038 segundos


In [9]:
def get_emails(connection, cursor):
    try:
        cursor.execute("""
            SELECT email FROM user_data.users
        """)
        rows = cursor.fetchall()
        # Acceder al primer elemento de cada tupla
        emails = [row[0] for row in rows]
        for email in emails:
            print(email)
    except Exception as e:
        print(f"Error al ejecutar la funcion: {e}")
        return []
    
if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        medir_tiempo_ejecucion(get_emails, connection, cursor)
    finally:
        cursor.close()
        connection.close()

Conexion exitosa a MySQL
antonlago04@example.com
spaladini1@who.int
csousa2@imgur.com
pharvett3@uiuc.edu
dseeler4@usda.gov
rwilcockes5@chicagotribune.com
tquilkin6@usatoday.com
mmion7@issuu.com
agobel8@yahoo.co.jp
npaulich9@wikispaces.com
oschneidara@storify.com
mkeavenyb@omniture.com
bambrusc@boston.com
dporsond@seattletimes.com
tablee@disqus.com
bmeadef@elegantthemes.com
twhillockg@oracle.com
akobierieckih@tuttocitta.it
tsteeli@blinklist.com
draddonj@blogs.com
Tiempo de ejecucion: -0.00045 segundos
